In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://09991c9b5d22:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-95f5070f-c241-44ec-a7cb-d8e0ca357485;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
from spark_utils import SparkUtils

In [3]:
schema = [
    ("from", "string"),
    ("to", "string")
]
spark_schema = SparkUtils.generate_schema(schema)
#i know this is not a good practice when it comes to pyspark i shoudnt direct to a single document, 
# more so when its supposed to take a lot more than just one document
df = spark.read.option("header", True).schema(spark_schema).csv("/opt/spark/work-dir/data/connections.csv")

string
string


In [4]:
df1 = df.dropDuplicates(['from', 'to'])

In [5]:
#df1.show(truncate=False)
finaldf = df1

In [6]:
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame

In [7]:
#i drop duplicates twice, so the question is which one would take less time
#Vertices DataFrame:
vert_df1 = finaldf.select(
    col("from").alias("link")
)
vert_df2 = finaldf.select(
    col("to").alias("link")
)
vertices_df = reduce(DataFrame.unionAll, [vert_df1, vert_df2]).dropDuplicates(["link"])

In [11]:
print(vertices_df.count())
vertices_df2 = vertices_df.dropna()
print(vertices_df2.count())

16622


16621


In [12]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"


vertices_df2.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":URL") \
  .option("node.keys", "link") \
  .save()

print(f"{vertices_df.count()} URL wrote in Neo4j")


16622 URL wrote in Neo4j
